In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [3]:
path_data = "/content/drive/MyDrive/CTM_Hien-main/Data/"
path_result = "/content/drive/MyDrive/CTM_Hien-main/Result/"
path_model = "/content/drive/MyDrive/CTM_Hien-main/Model/"

In [4]:
def twoTupleDic3():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
             AA_dict[i+j+jj] = numm
             numm += 1
    return AA_dict
def twoTupleDic2():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          AA_dict[i+j] = numm
          numm += 1
    return AA_dict

def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [5]:
# Tải dữ liệu train model
import pandas as pd
file_train = "train_data_3speaces_31.csv"
df_train=pd.read_csv(path_data + file_train, delimiter= ',')

texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data input Word2vec
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())

MAX_SEQUENCE_LENGTH= len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain = lb.fit_transform(ytrain)
ytrain= to_categorical(ytrain)
ytrain.shape


(6746, 2)

**Define model**

In [6]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results



In [7]:
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')
result_test =path_result +"Result_KD.txt"
num_folds = 5
TIME_STEPS = 33
INPUT_SIZE = 300


In [8]:
#  Cross validation Knowlege distillation model
kf = KFold(n_splits=num_folds, shuffle=True)

X = Xtrain
Y = ytrain
model_teacher = models.load_model(path_model + 'model_teacher.h5')
fold_idx =1
for train_index, test_index in kf.split(X,Y):
    X_train, X_test =X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
# Create the student
    print("Start train Fold ", fold_idx)
    student = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.dropout(0.2),
        layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        layers.Dropout(0.2),
        #layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="student",
    )
    student_scratch = keras.models.clone_model(student)
    distiller = Distiller(student=student, teacher= model_teacher)
    distiller.compile(
          student_loss_fn= keras.losses.CategoricalCrossentropy(from_logits=True),
          #student_loss_fn= keras.losses.BinaryCrossentropy(from_logits=True),
          optimizer=keras.optimizers.Adam(learning_rate = 0.0001),
          metrics=[
                  "categorical_accuracy",
                  "AUC",
                  ], # , f1
          distillation_loss_fn=keras.losses.KLDivergence(),
          alpha=0.1,
          temperature=10
    )
    history_KD = distiller.fit(X_train, Y_train, batch_size=16, validation_data =(X_test, Y_test), epochs=100, callbacks=[checkpoint],verbose=1)
    result2 = distiller.evaluate(X_test, Y_test)#, validation_data=(X_test,Y_test), shuffle = True,callbacks=[checkpoint],verbose=1)
    f = open(result_test, 'a+', encoding='UTF-8')
    f.write("\n KD_BiLSTM fold: \n")
    f.write(str(fold_idx))
    s = str(result2)
    f.write(s)
    f.close()


    ypred = student.predict(X_test)
    ypred =np.argmax(ypred,axis =1)

    ytest_true = np.argmax(Y_test,axis =1)

    result22 = confusion_matrix(ytest_true,ypred)
    print(result22)
    f = open(result_test, 'a+', encoding='UTF-8')
    f.write("\n Fold Confusion_matric: \n ")
    s = str(result22)
    f.write(s)
    f.close()
    fold_idx = fold_idx +1

Start train Fold  1
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:593: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:592: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


338/338 ━━━━━━━━━━━━━━━━━━━━ 32s 81ms/step - AUC: 0.6439 - categorical_accuracy: 0.6051 - distillation_loss: 0.0098 - loss: 0.5000 - student_loss: 0.6260 - val_AUC: 0.7846 - val_categorical_accuracy: 0.7111 - val_loss: 0.5000 - val_student_loss: 0.5050
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 27s 81ms/step - AUC: 0.7769 - categorical_accuracy: 0.7098 - distillation_loss: 0.0022 - loss: 0.5000 - student_loss: 0.5737 - val_AUC: 0.7945 - val_categorical_accuracy: 0.7252 - val_loss: 0.5000 - val_student_loss: 0.4206
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 39s 75ms/step - AUC: 0.7831 - categorical_accuracy: 0.7232 - distillation_loss: 0.0016 - loss: 0.5000 - student_loss: 0.5645 - val_AUC: 0.7953 - val_categorical_accuracy: 0.7244 - val_loss: 0.5000 - val_student_loss: 0.4185
Epoch 4/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - AUC: 0.7814 - categorical_accuracy: 0.7202 - distillation_loss: 0.0014 - loss: 0.5000 - student_loss: 0.5638 - val_AUC: 0.8076 - val_categorical_accuracy: 0.740

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:593: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:592: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


338/338 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - AUC: 0.6446 - categorical_accuracy: 0.6091 - distillation_loss: 0.0094 - loss: 0.5000 - student_loss: 0.6185 - val_AUC: 0.7910 - val_categorical_accuracy: 0.7228 - val_loss: 0.5000 - val_student_loss: 0.7935
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 26s 77ms/step - AUC: 0.7703 - categorical_accuracy: 0.6981 - distillation_loss: 0.0020 - loss: 0.5000 - student_loss: 0.5742 - val_AUC: 0.8011 - val_categorical_accuracy: 0.7391 - val_loss: 0.5000 - val_student_loss: 0.8032
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 24s 72ms/step - AUC: 0.7812 - categorical_accuracy: 0.7191 - distillation_loss: 0.0015 - loss: 0.5000 - student_loss: 0.5654 - val_AUC: 0.8013 - val_categorical_accuracy: 0.7487 - val_loss: 0.5000 - val_student_loss: 0.6987
Epoch 3: early stopping
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - AUC: 0.7982 - categorical_accuracy: 0.7523 - loss: 0.5000 - student_loss: 0.5530
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
[[516 144]
 [195 494]]
Start t

...

338/338 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - AUC: 0.6408 - categorical_accuracy: 0.5931 - distillation_loss: 0.0094 - loss: 0.5000 - student_loss: 0.6164 - val_AUC: 0.7865 - val_categorical_accuracy: 0.7168 - val_loss: 0.5000 - val_student_loss: 0.5075
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 38s 68ms/step - AUC: 0.7763 - categorical_accuracy: 0.7150 - distillation_loss: 0.0019 - loss: 0.5000 - student_loss: 0.5717 - val_AUC: 0.7896 - val_categorical_accuracy: 0.7205 - val_loss: 0.5000 - val_student_loss: 0.5534
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 44s 77ms/step - AUC: 0.7790 - categorical_accuracy: 0.7163 - distillation_loss: 0.0016 - loss: 0.5000 - student_loss: 0.5616 - val_AUC: 0.7949 - val_categorical_accuracy: 0.7198 - val_loss: 0.5000 - val_student_loss: 0.5155
Epoch 3: early stopping
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - AUC: 0.8089 - categorical_accuracy: 0.7373 - loss: 0.5000 - student_loss: 0.5562
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
[[511 187]
 [191 460]]
Start t

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:593: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:592: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


338/338 ━━━━━━━━━━━━━━━━━━━━ 37s 98ms/step - AUC: 0.6276 - categorical_accuracy: 0.5798 - distillation_loss: 0.0096 - loss: 0.5000 - student_loss: 0.6168 - val_AUC: 0.7590 - val_categorical_accuracy: 0.6998 - val_loss: 0.5000 - val_student_loss: 0.7084
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 26s 78ms/step - AUC: 0.7888 - categorical_accuracy: 0.7208 - distillation_loss: 0.0022 - loss: 0.5000 - student_loss: 0.5658 - val_AUC: 0.7684 - val_categorical_accuracy: 0.7146 - val_loss: 0.5000 - val_student_loss: 0.7773
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - AUC: 0.7955 - categorical_accuracy: 0.7344 - distillation_loss: 0.0016 - loss: 0.5000 - student_loss: 0.5569 - val_AUC: 0.7762 - val_categorical_accuracy: 0.7153 - val_loss: 0.5000 - val_student_loss: 0.7314
Epoch 3: early stopping
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - AUC: 0.7869 - categorical_accuracy: 0.7297 - loss: 0.5000 - student_loss: 0.5812
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
[[496 174]
 [210 469]]
Start t

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:593: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:592: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


338/338 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - AUC: 0.6306 - categorical_accuracy: 0.5819 - distillation_loss: 0.0094 - loss: 0.5000 - student_loss: 0.6149 - val_AUC: 0.7814 - val_categorical_accuracy: 0.7064 - val_loss: 0.5000 - val_student_loss: 0.3990
Epoch 2/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 26s 77ms/step - AUC: 0.7793 - categorical_accuracy: 0.7160 - distillation_loss: 0.0020 - loss: 0.5000 - student_loss: 0.5680 - val_AUC: 0.7896 - val_categorical_accuracy: 0.7317 - val_loss: 0.5000 - val_student_loss: 0.4569
Epoch 3/100
338/338 ━━━━━━━━━━━━━━━━━━━━ 42s 79ms/step - AUC: 0.7992 - categorical_accuracy: 0.7348 - distillation_loss: 0.0015 - loss: 0.5000 - student_loss: 0.5627 - val_AUC: 0.7930 - val_categorical_accuracy: 0.7309 - val_loss: 0.5000 - val_student_loss: 0.3989
Epoch 3: early stopping
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - AUC: 0.8215 - categorical_accuracy: 0.7530 - loss: 0.5000 - student_loss: 0.5536
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
[[490 182]
 [181 496]]
